# Gmail Tool 테스트
get_gmail_tools()로 툴 목록 확인, 개별 툴 동작, LangGraph agent 연동까지 순서대로 테스트합니다.

## 0. load env

In [ ]:
import os
import sys

from dotenv import load_dotenv, find_dotenv

# Load environment variables
dotenv_path = find_dotenv()
print(f"Target .env path: {dotenv_path}")
loaded = load_dotenv(dotenv_path, override=True)
print(f"Loading successful: {loaded}")

# Add project root to path
sys.path.append(os.path.abspath('../../../../..'))

## 1. 툴 목록 확인

In [ ]:
from app.agents.tools.gmail import get_gmail_tools

gmail_tools = get_gmail_tools()

print(f"✅ 툴 개수: {len(gmail_tools)}")
for tool in gmail_tools:
    print(f"  - name: {tool.name}")
    print(f"    desc: {tool.description[:60]}...")

## 2. 메일 검색 테스트 (GmailSearch)

In [ ]:
# 툴 이름으로 원하는 툴만 꺼내기
search_tool = next(t for t in gmail_tools if t.name == "search_gmail")

# 최근 메일 3개 검색
result = search_tool.invoke({"query": "in:inbox", "max_results": 3})
print(result)

## 3. 메일 초안 작성 테스트 (GmailCreateDraft)

In [ ]:
draft_tool = next(t for t in gmail_tools if t.name == "create_gmail_draft")

result = draft_tool.invoke({
    "to": ["test@example.com"],
    "subject": "[테스트] Gmail Tool 동작 확인",
    "message": "안녕하세요. LangGraph Gmail Tool 테스트 메일입니다."
})
print(result)

## 4. LangGraph Agent 연동 테스트

In [ ]:
from typing import Annotated, TypedDict

from langchain_core.messages import AnyMessage, HumanMessage
from langgraph.graph import END, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import tools_condition

from app.agents.core.agent_factory import build_agent, create_tool_node_with_fallback


# ── State ────────────────────────────────────────────────────────────────────
class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]


# ── Agent & Tool Node ────────────────────────────────────────────────────────
tools = gmail_tools  # 위 셀에서 가져온 gmail_tools 재사용

agent = build_agent(
    tools=tools,
    system_prompt="당신은 이메일 어시스턴트입니다. Gmail 툴을 사용해 메일 검색, 초안 작성, 전송을 도와줍니다.",
)

tool_node = create_tool_node_with_fallback(tools)

# ── Graph ────────────────────────────────────────────────────────────────────
workflow = StateGraph(State)
workflow.add_node("agent", agent)
workflow.add_node("tools", tool_node)
workflow.set_entry_point("agent")
workflow.add_conditional_edges(
    "agent",
    tools_condition,
    {
        "tools": "tools",
        END: END
    }
)
workflow.add_edge("tools", "agent")

app = workflow.compile()
print("✅ Graph 컴파일 완료")

# ── 실행: 메일 검색 ───────────────────────────────────────────────────────────
query = "test@gmail.com 에게 '안녕하세요' 라는 제목으로 인사 메일 초안 작성해주고"
async for chunk in app.astream(
        {"messages": [HumanMessage(content=query)]},
        stream_mode="values"
):
    chunk["messages"][-1].pretty_print()

In [ ]:
# Graph 시각화
from IPython.display import Image

Image(app.get_graph().draw_mermaid_png())